# Gaussian Iterative Stockholder Analysis (GISA) method

In [1]:
import logging

from scipy.optimize import least_squares
from setup import *

from horton_part import GaussianISAWPart

## Quadprog Solver

The problem of this solver is that the initials values are set to zeros for all pro-atom parameters and no threshold setting is availiable.

### Using API

In [2]:
mol, grid, rho = prepare_grid_and_dens("data/h2o.fchk")


def quadprog_solver():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = "quadprog"
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)


quadprog_solver()

The number of electrons: 10.000003764139395
Coordinates of the atoms 
 [[ 0.     0.     0.224]
 [-0.     1.457 -0.896]
 [-0.    -1.457 -0.896]]
Atomic numbers of the atom 
 [8 1 1]

Information of integral grids.
--------------------------------------------------------------------------------
Compute local grids ...
Grid size of molecular grid: 18460
************************************ Atom 0 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 8252
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18
          18 18 18 18 18 18 26 26 26 26 38 38 38 38 38 50 50 50 50 50
          86 86 110 110 110 110 170 194 194 434 590 590 434 434 434 302 302 302 194 194
          170 110 110 110 110 110 110 110 110 110 110 110 86 50 50 18 18 18 18 18
          
************************************ At

### Customized Implementation

In [3]:
import quadprog

from horton_part import check_pro_atom_parameters


def _prepare_quantities(bs_funcs, rho, propars, weights, alphas):
    nprim = len(propars)
    G = (
        2
        / np.pi**1.5
        * (alphas[:, None] * alphas[None, :]) ** 1.5
        / (alphas[:, None] + alphas[None, :]) ** 1.5
    )
    G = (G + G.T) / 2

    # a = np.zeros(nprim, float)
    # for k in range(nprim):
    #     a[k] = 2 * np.einsum("i,i,i", weights, bs_funcs[k], rho)
    a = 2 * np.einsum("i,ni,i->n", weights, bs_funcs, rho)

    # Construct linear equality or inequality constraints
    C = np.zeros([nprim, nprim + 1])
    # First column : corresponds to the EQUALITY constraint sum_{k=1..Ka} c_(a,k)= N_a
    C[:, 0] = np.ones(nprim)
    # Other K_a columns : correspond to the INEQUALITY constraints c_(a,k) >=0
    C[0:nprim, 1 : (nprim + 1)] = np.identity(nprim)
    b = np.zeros(nprim + 1)
    # First coefficient : corresponds to the EQUALITY constraint sum_{k=1..Ka} c_(a,k) = N_a
    pop = np.einsum("i,i", weights, rho)
    b[0] = pop
    return G, a, C, b


def opt_propars_qp_quadprog(bs_funcs, rho, propars, points, weights, alphas, threshold):
    r"""
    Optimize parameters for proatom density functions using quadratic programming.

    .. math::

        N_{Ai} = \int \rho_A(r) \frac{\rho_{Ai}^0(r)}{\rho_A^0(r)} dr


        G = \frac{1}{2} c^T S c - c^T b

        S = 2 \int \zeta(\vec{r}) \zeta(\vec{r}) d\vec{r}
        = \frac{2}{\pi \sqrt{\pi}} \frac{(\alpha_k \alpha_l)^{3/2}}{(\alpha_k + \alpha_l)^{3/2}}

        b = 2 * \int \zeta(\vec{r}) \rho_a(\vec{r}) d\vec{r}

    Parameters
    ----------
    bs_funcs : 2D np.ndarray
        Basis functions array with shape (M, N), where 'M' is the number of basis functions
        and 'N' is the number of grid points.
    rho : 1D np.ndarray
        Spherically-averaged atomic density as a function of radial distance, with shape (N,).
    propars : 1D np.ndarray
        Pro-atom parameters with shape (M). 'M' is the number of basis functions.
    points : 1D np.ndarray
        Radial coordinates of grid points, with shape (N,).
    weights : 1D np.ndarray
        Weights for integration, including the angular part (4πr²), with shape (N,).
    threshold : float
        Convergence threshold for the iterative process.
    alphas : 1D np.ndarray
        The Gaussian exponential coefficients.
    threshold : float
        The convergence threshold of the optimization method.

    Returns
    -------
    1D np.ndarray
        Optimized proatom parameters.

    Raises
    ------
    RuntimeError
        If the inner iteration does not converge.


    """
    G, a, C, b = _prepare_quantities(bs_funcs, rho, propars, weights, alphas)
    result = quadprog.solve_qp(G=G, a=a, C=C, b=b, meq=1)[0]
    check_pro_atom_parameters(result, total_population=float(b[0]))
    return result

Now we can call the customized solver through follows:

In [4]:
def customized_quadprog_solver():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = opt_propars_qp_quadprog
    kwargs["inner_threshold"] = 1e-8
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)


customized_quadprog_solver()


Information of integral grids.
--------------------------------------------------------------------------------
Compute local grids ...
Grid size of molecular grid: 18460
************************************ Atom 0 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 8252
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18
          18 18 18 18 18 18 26 26 26 26 38 38 38 38 38 50 50 50 50 50
          86 86 110 110 110 110 170 194 194 434 590 590 434 434 434 302 302 302 194 194
          170 110 110 110 110 110 110 110 110 110 110 110 86 50 50 18 18 18 18 18
          
************************************ Atom 1 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 5104
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 

/Users/runner/work/horton-part/horton-part/src/horton_part/gisa.py:185: UserWarning: Customized solver is used, the argument `inner_threshold` is not used.
  warnings.warn("Customized solver is used, the argument `inner_threshold` is not used.")


       35   1.02925e-05   4.07788e-02
       36   8.63147e-06   4.07756e-02
       37   7.24041e-06   4.07729e-02
       38   6.07489e-06   4.07706e-02
       39   5.09794e-06   4.07687e-02
       40   4.27879e-06   4.07671e-02
       41   3.59174e-06   4.07658e-02
       42   3.01536e-06   4.07647e-02
       43   2.53171e-06   4.07637e-02
       44   2.12581e-06   4.07630e-02
       45   1.78511e-06   4.07623e-02
       46   1.49911e-06   4.07617e-02
       47   1.25898e-06   4.07613e-02
       48   1.05737e-06   4.07609e-02
       49   8.88068e-07   4.07606e-02

Computing atomic populations.
Computing atomic charges.
Computing density decomposition for atom 0
Computing density decomposition for atom 1
Computing density decomposition for atom 2
Computing cartesian and pure AIM multipoles and radial AIM moments.
Storing proatom density spline for atom 0.
Storing proatom density spline for atom 1.
Storing proatom density spline for atom 2.
charges:
[-0.863  0.432  0.432]
cartesian multi

## CVXOPT Solver
Another sovler implemented in `CVXOPT` can also be applied.

### Using API

In [5]:
def cvxopt_solver():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = "cvxopt"
    kwargs["solver_options"] = {"eps_rel": 1e-8}
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)
    opt_propars = part._cache["propars"]
    print("Difference between total electrons:")
    print(grid.integrate(rho) - np.sum(opt_propars))


cvxopt_solver()


Information of integral grids.
--------------------------------------------------------------------------------
Compute local grids ...
Grid size of molecular grid: 18460
************************************ Atom 0 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 8252
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18
          18 18 18 18 18 18 26 26 26 26 38 38 38 38 38 50 50 50 50 50
          86 86 110 110 110 110 170 194 194 434 590 590 434 434 434 302 302 302 194 194
          170 110 110 110 110 110 110 110 110 110 110 110 86 50 50 18 18 18 18 18
          
************************************ Atom 1 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 5104
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 

### Customized Implementation

In [6]:
import cvxopt


def opt_propars_qp_cvxopt(bs_funcs, rho, propars, points, weights, alphas, threshold):
    """
    Optimize pro-atom parameters using quadratic-programming implemented in the `CVXOPT` package.

    Parameters
    ----------
    bs_funcs : 2D np.ndarray
        Basis functions array with shape (M, N), where 'M' is the number of basis functions
        and 'N' is the number of grid points.
    rho : 1D np.ndarray
        Spherically-averaged atomic density as a function of radial distance, with shape (N,).
    propars : 1D np.ndarray
        Pro-atom parameters with shape (M). 'M' is the number of basis functions.
    points : 1D np.ndarray
        Radial coordinates of grid points, with shape (N,).
    weights : 1D np.ndarray
        Weights for integration, including the angular part (4πr²), with shape (N,).
    threshold : float
        Convergence threshold for the iterative process.
    alphas : 1D np.ndarray
        The Gaussian exponential coefficients.
    threshold : float
        The convergence threshold of the optimization method.

    Returns
    -------
    1D np.ndarray
        Optimized proatom parameters.

    Raises
    ------
    RuntimeError
        If the inner iteration does not converge.

    """
    nprim, npt = bs_funcs.shape

    S = (
        2
        / np.pi**1.5
        * (alphas[:, None] * alphas[None, :]) ** 1.5
        / (alphas[:, None] + alphas[None, :]) ** 1.5
    )
    P = cvxopt.matrix(S)

    vec_b = np.zeros((nprim, 1), float)
    for k in range(nprim):
        vec_b[k] = 2 * np.einsum("i,i", weights * bs_funcs[k], rho)
    q = -cvxopt.matrix(vec_b)

    # Linear inequality constraints
    G = cvxopt.matrix(0.0, (nprim, nprim))
    G[:: nprim + 1] = -1.0
    h = cvxopt.matrix(0.0, (nprim, 1))

    # Linear equality constraints
    A = cvxopt.matrix(1.0, (1, nprim))
    pop = np.einsum("i,i", weights, rho)
    b = cvxopt.matrix(pop, (1, 1))

    # initial_values = cvxopt.matrix(np.array([1.0] * nprim).reshape((nprim, 1)))
    opt_CVX = cvxopt.solvers.qp(
        P,
        q,
        G,
        h,
        A,
        b,
        # initvals=propars,
        initvals=np.zeros_like(propars),
        options={"feastol": threshold, "show_progress": 0},
        # options={"show_progress": log.do_medium, "feastol": threshold},
    )
    new_propars = np.asarray(opt_CVX["x"]).flatten()
    check_pro_atom_parameters(new_propars, total_population=float(pop))
    return new_propars

In [7]:
def customized_cvxopt_solver():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = opt_propars_qp_cvxopt
    kwargs["inner_threshold"] = 1e-8
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)
    opt_propars = part._cache["propars"]
    print("Difference between total electrons:")
    print(grid.integrate(rho) - np.sum(opt_propars))


customized_cvxopt_solver()


Information of integral grids.
--------------------------------------------------------------------------------
Compute local grids ...
Grid size of molecular grid: 18460
************************************ Atom 0 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 8252
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18
          18 18 18 18 18 18 26 26 26 26 38 38 38 38 38 50 50 50 50 50
          86 86 110 110 110 110 170 194 194 434 590 590 434 434 434 302 302 302 194 194
          170 110 110 110 110 110 110 110 110 110 110 110 86 50 50 18 18 18 18 18
          
************************************ Atom 1 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 5104
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 

## ECOS Sovler

In [8]:
def ecos_solver():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = "ecos"
    kwargs["solver_options"] = {"reltol": 1e-10}
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)
    opt_propars = part._cache["propars"]
    print("Difference between total electrons:")
    print(grid.integrate(rho) - np.sum(opt_propars))


ecos_solver()


Information of integral grids.
--------------------------------------------------------------------------------
Compute local grids ...
Grid size of molecular grid: 18460
************************************ Atom 0 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 8252
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18
          18 18 18 18 18 18 26 26 26 26 38 38 38 38 38 50 50 50 50 50
          86 86 110 110 110 110 170 194 194 434 590 590 434 434 434 302 302 302 194 194
          170 110 110 110 110 110 110 110 110 110 110 110 86 50 50 18 18 18 18 18
          
************************************ Atom 1 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 5104
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/qpsolvers/solvers/ecos_.py:126: UserWarning: warm-start values are ignored by this wrapper
  warnings.warn("warm-start values are ignored by this wrapper")


       31   2.06940e-05   4.07939e-02
       32   1.73437e-05   4.07871e-02
       33   1.44613e-05   4.07813e-02
       34   1.21783e-05   4.07765e-02
       35   1.01895e-05   4.07725e-02
       36   8.55436e-06   4.07692e-02
       37   7.22805e-06   4.07664e-02
       38   6.02223e-06   4.07640e-02
       39   5.01993e-06   4.07621e-02
       40   4.28507e-06   4.07604e-02
       41   3.59616e-06   4.07590e-02
       42   2.98675e-06   4.07578e-02
       43   2.53450e-06   4.07569e-02
       44   2.16147e-06   4.07560e-02
       45   1.77456e-06   4.07554e-02
       46   1.50009e-06   4.07548e-02
       47   1.25209e-06   4.07543e-02
       48   1.07946e-06   4.07539e-02
       49   9.03533e-07   4.07535e-02

Computing atomic populations.
Computing atomic charges.
Computing density decomposition for atom 0
Computing density decomposition for atom 1
Computing density decomposition for atom 2
Computing cartesian and pure AIM multipoles and radial AIM moments.
Storing proatom density 

## OSQP Solver

In [9]:
def osqp_solver():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = "osqp"
    kwargs["solver_options"] = {"eps_rel": 1e-14}
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)
    opt_propars = part._cache["propars"]
    print("Difference between total electrons:")
    print(grid.integrate(rho) - np.sum(opt_propars))


osqp_solver()


Information of integral grids.
--------------------------------------------------------------------------------
Compute local grids ...
Grid size of molecular grid: 18460
************************************ Atom 0 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 8252
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18
          18 18 18 18 18 18 26 26 26 26 38 38 38 38 38 50 50 50 50 50
          86 86 110 110 110 110 170 194 194 434 590 590 434 434 434 302 302 302 194 194
          170 110 110 110 110 110 110 110 110 110 110 110 86 50 50 18 18 18 18 18
          
************************************ Atom 1 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 5104
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted P to scipy.sparse.csc.csc_matrix
For best performance, build P as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted G to scipy.sparse.csc.csc_matrix
For best performance, build G as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted A to scipy.sparse.csc.csc_matrix
For best performance, build A as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/Users/runner/work/horton-part/horton-part/src/horton_part/utils.py:571: UserWarning: The sum of pro-atom parameter

       29   3.06094e-05   4.10651e-02
       30   2.56442e-05   4.10556e-02
       31   2.14997e-05   4.10477e-02
       32   1.80360e-05   4.10411e-02
       33   1.51383e-05   4.10356e-02
       34   1.27117e-05   4.10310e-02
       35   1.06782e-05   4.10271e-02
       36   8.97286e-06   4.10239e-02
       37   7.54196e-06   4.10212e-02
       38   6.34074e-06   4.10189e-02
       39   5.33189e-06   4.10170e-02
       40   4.48432e-06   4.10154e-02
       41   3.77202e-06   4.10141e-02
       42   3.17325e-06   4.10129e-02
       43   2.66980e-06   4.10120e-02
       44   2.24642e-06   4.10112e-02
       45   1.89033e-06   4.10105e-02
       46   1.59077e-06   4.10099e-02
       47   1.33876e-06   4.10095e-02
       48   1.12673e-06   4.10091e-02
       49   9.48307e-07   4.10087e-02

Computing atomic populations.
Computing atomic charges.
Computing density decomposition for atom 0
Computing density decomposition for atom 1
Computing density decomposition for atom 2
Computing cartes

/Users/runner/work/horton-part/horton-part/src/horton_part/utils.py:571: UserWarning: The sum of pro-atom parameters is not equal to atomic population.The difference is -0.0001404432451957982
  warnings.warn(
/Users/runner/work/horton-part/horton-part/src/horton_part/utils.py:571: UserWarning: The sum of pro-atom parameters is not equal to atomic population.The difference is -0.00014044324519590923
  warnings.warn(
/Users/runner/work/horton-part/horton-part/src/horton_part/utils.py:571: UserWarning: The sum of pro-atom parameters is not equal to atomic population.The difference is -0.00014044702432847167
  warnings.warn(
/Users/runner/work/horton-part/horton-part/src/horton_part/utils.py:571: UserWarning: The sum of pro-atom parameters is not equal to atomic population.The difference is -0.0001404470243285827
  warnings.warn(
/Users/runner/work/horton-part/horton-part/src/horton_part/utils.py:571: UserWarning: The sum of pro-atom parameters is not equal to atomic population.The differe

Storing proatom density spline for atom 0.
Storing proatom density spline for atom 1.
Storing proatom density spline for atom 2.
charges:
[-0.865  0.432  0.432]
cartesian multipoles:
[[-0.865  0.    -0.     0.114 -5.316 -0.    -0.    -4.722 -0.    -5.017 -0.     0.    -0.037  0.     0.    -0.     0.     0.326  0.     0.162]
 [ 0.432  0.     0.023 -0.001 -0.308 -0.    -0.    -0.281  0.002 -0.287  0.     0.013  0.003  0.     0.     0.     0.038  0.009  0.015  0.02 ]
 [ 0.432  0.    -0.023 -0.001 -0.308  0.     0.    -0.281 -0.002 -0.287  0.    -0.013  0.003  0.    -0.     0.    -0.038  0.009 -0.015  0.02 ]]
Difference between total electrons:
0.0003009687201434019


More supported sovlers can be found in [`qpsovlers`](https://qpsolvers.github.io/qpsolvers/supported-solvers.html) package.

## Customized SLSQP Solver

A equivalent method is using `minimization` with the 'SLSQP' sovler implemented in `SciPy` package. 

In [10]:
def opt_propars_qp_slsqp(bs_funcs, rho, propars, points, weights, alphas, threshold):
    r"""
    Optimize parameters for proatom density functions using quadratic programming.

    .. math::

        N_{Ai} = \int \rho_A(r) \frac{\rho_{Ai}^0(r)}{\rho_A^0(r)} dr


        G = \frac{1}{2} c^T S c - c^T b

        S = 2 \int \zeta(\vec{r}) \zeta(\vec{r}) d\vec{r}
        = \frac{2}{\pi \sqrt{\pi}} \frac{(\alpha_k \alpha_l)^{3/2}}{(\alpha_k + \alpha_l)^{3/2}}

        b = 2 * \int \zeta(\vec{r}) \rho_a(\vec{r}) d\vec{r}

    Parameters
    ----------
    bs_funcs : 2D np.ndarray
        Basis functions array with shape (M, N), where 'M' is the number of basis functions
        and 'N' is the number of grid points.
    rho : 1D np.ndarray
        Spherically-averaged atomic density as a function of radial distance, with shape (N,).
    propars : 1D np.ndarray
        Pro-atom parameters with shape (M). 'M' is the number of basis functions.
    points : 1D np.ndarray
        Radial coordinates of grid points, with shape (N,).
    weights : 1D np.ndarray
        Weights for integration, including the angular part (4πr²), with shape (N,).
    threshold : float
        Convergence threshold for the iterative process.
    alphas : 1D np.ndarray
        The Gaussian exponential coefficients.
    threshold : float
        The convergence threshold of the optimization method.

    Returns
    -------
    1D np.ndarray
        Optimized proatom parameters.

    Raises
    ------
    RuntimeError
        If the inner iteration does not converge.


    """
    G, a, C, b = _prepare_quantities(bs_funcs, rho, propars, weights, alphas)
    meq = 1

    def _obj_func(x):
        return 0.5 * np.dot(x, G).dot(x) - np.dot(a, x)

    constraints = []
    if C is not None:
        constraints = [
            {
                "type": "eq" if i < meq else "ineq",
                "fun": lambda x, C=C, b=b, i=i: (np.dot(C.T, x) - b)[i],
            }
            for i in range(C.shape[1])
        ]

    result = minimize(
        _obj_func,
        x0=np.zeros(len(G)),
        method="SLSQP",
        constraints=constraints,
        tol=threshold,
        options={"maxiter": 2000},
    )
    return result.x

In [11]:
def customized_slsqp_solver():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = opt_propars_qp_cvxopt
    # the value is from the test file of quadprog package.
    kwargs["inner_threshold"] = 1e-10
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)
    opt_propars = part._cache["propars"]
    print("Difference between total electrons:")
    print(grid.integrate(rho) - np.sum(opt_propars))


customized_slsqp_solver()


Information of integral grids.
--------------------------------------------------------------------------------
Compute local grids ...
Grid size of molecular grid: 18460
************************************ Atom 0 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 8252
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18
          18 18 18 18 18 18 26 26 26 26 38 38 38 38 38 50 50 50 50 50
          86 86 110 110 110 110 170 194 194 434 590 590 434 434 434 302 302 302 194 194
          170 110 110 110 110 110 110 110 110 110 110 110 86 50 50 18 18 18 18 18
          
************************************ Atom 1 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 5104
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 

## Customized Least-Square Sovler

Users can easily apply customized solvers. The first step involves defining a customized solver, for example, `customized_solver`. The corresponding arguments and their meanings are as follows:

In [12]:
def opt_propars_lstsq(bs_funcs, rho, propars, points, weights, alphas, threshold):
    r"""
    Optimize pro-atom parameters using quadratic-programming implemented in the `CVXOPT` package.

    .. math::

        N_{Ai} = \int \rho_A(r) \frac{\rho_{Ai}^0(r)}{\rho_A^0(r)} dr

        G = \frac{1}{2} c^T S c - c^T b

        S = 2 \int \zeta(\vec{r}) \zeta(\vec{r}) d\vec{r}
        = \frac{2}{\pi \sqrt{\pi}} \frac{(\alpha_k \alpha_l)^{3/2}}{(\alpha_k + \alpha_l)^{3/2}}

        b = \int \zeta(\vec{r}) \rho_a(\vec{r}) d\vec{r}

    Parameters
    ----------
    bs_funcs : 2D np.ndarray
        Basis functions array with shape (M, N), where 'M' is the number of basis functions
        and 'N' is the number of grid points.
    rho : 1D np.ndarray
        Spherically-averaged atomic density as a function of radial distance, with shape (N,).
    propars : 1D np.ndarray
        Pro-atom parameters with shape (M). 'M' is the number of basis functions.
    points : 1D np.ndarray
        Radial coordinates of grid points, with shape (N,).
    weights : 1D np.ndarray
        Weights for integration, including the angular part (4πr²), with shape (N,).
    threshold : float
        Convergence threshold for the iterative process.
    alphas : 1D np.ndarray
        The Gaussian exponential coefficients.
    threshold : float
        The convergence threshold of the optimization method.

    Returns
    -------
    1D np.ndarray
        Optimized proatom parameters.

    Raises
    ------
    RuntimeError
        If the inner iteration does not converge.

    """

    def _obj_func(x):
        pro = np.sum(x[:, None] * bs_funcs, axis=0)
        return np.sqrt(weights) * np.abs(pro - rho)

    result = least_squares(
        _obj_func, x0=np.zeros_like(propars), bounds=(0, np.inf), ftol=threshold
    )
    return result.x

Next, we need to pass this customized solver to our partitioning method.

In [13]:
def customized_lstsq_solver():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = opt_propars_lstsq
    kwargs["inner_threshold"] = 1e-14
    part = GaussianISAWPart(**kwargs)
    part.do_all()

    print_results(part)
    opt_propars = part._cache["propars"]
    print("Difference between total electrons:")
    print(grid.integrate(rho) - np.sum(opt_propars))


customized_lstsq_solver()


Information of integral grids.
--------------------------------------------------------------------------------
Compute local grids ...
Grid size of molecular grid: 18460
************************************ Atom 0 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 8252
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18
          18 18 18 18 18 18 26 26 26 26 38 38 38 38 38 50 50 50 50 50
          86 86 110 110 110 110 170 194 194 434 590 590 434 434 434 302 302 302 194 194
          170 110 110 110 110 110 110 110 110 110 110 110 86 50 50 18 18 18 18 18
          
************************************ Atom 1 ************************************
|-- Local grid size: 18460
|-- Atom grid size: 5104
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 

The result of the least squares method differs slightly from the quadratic programming method due to the constraints. We can observe that the deviation in the total electrons obtained by the least squares method is larger than that obtained by the quadratic programming method.